In [ ]:
# downloading the dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-23 11:16:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.61MB/s    in 0.7s    

2025-06-23 11:16:18 (1.61 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# reading the file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [6]:
# taking a look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Vocab Size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab Size:  65


In [10]:
# create a mapping from chars to ints
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encode takes a str and returns list of int
decode = lambda l: ''.join([itos[i] for i in l]) # decode takes list of int and converts it to str

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [11]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
# lets now split the data into train and val
n = int(0.9 * len(data)) # 90% train data and 10% val data
train_data = data[:n]
val_data = data[n:]

In [ ]:
# we will send chunks of data into the transformer and not the entire text
block_size = 8
train_data[:block_size+1] # block_size + 1 becuase it 9 characters have 8 examples 
                          # when given 18 model will predict 47, and when given [18, 47] model will predict 56 and so on..

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [19]:
# the 8 examples are
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{t+1}: when input is {context} the target is {target}")

1: when input is tensor([18]) the target is 47
2: when input is tensor([18, 47]) the target is 56
3: when input is tensor([18, 47, 56]) the target is 57
4: when input is tensor([18, 47, 56, 57]) the target is 58
5: when input is tensor([18, 47, 56, 57, 58]) the target is 1
6: when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
7: when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
8: when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [23]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [29]:
# to check what is input and what is output
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when context is {context.tolist()} the target is {target}")

when context is [57] the target is 1
when context is [57, 1] the target is 46
when context is [57, 1, 46] the target is 47
when context is [57, 1, 46, 47] the target is 57
when context is [57, 1, 46, 47, 57] the target is 1
when context is [57, 1, 46, 47, 57, 1] the target is 50
when context is [57, 1, 46, 47, 57, 1, 50] the target is 53
when context is [57, 1, 46, 47, 57, 1, 50, 53] the target is 60
when context is [1] the target is 58
when context is [1, 58] the target is 46
when context is [1, 58, 46] the target is 43
when context is [1, 58, 46, 43] the target is 56
when context is [1, 58, 46, 43, 56] the target is 43
when context is [1, 58, 46, 43, 56, 43] the target is 1
when context is [1, 58, 46, 43, 56, 43, 1] the target is 41
when context is [1, 58, 46, 43, 56, 43, 1, 41] the target is 39
when context is [17] the target is 26
when context is [17, 26] the target is 15
when context is [17, 26, 15] the target is 17
when context is [17, 26, 15, 17] the target is 10
when context is

In [ ]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits from the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, idx, targets=None):
        # idx and target are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # B,T,C -> (Batch,Training,Vocab_Size)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, target=targets)
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [42]:
m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

# testing the generate func
test_idx = torch.zeros((1,1), dtype=torch.long)
generate_op = m.generate(test_idx, max_new_tokens=100)[0].tolist()
print(generate_op)
print(decode(generate_op))

torch.Size([32, 65]) tensor(4.3431, grad_fn=<NllLossBackward0>)
[0, 19, 40, 55, 10, 30, 44, 12, 34, 53, 4, 19, 20, 39, 3, 13, 55, 61, 43, 1, 13, 37, 8, 57, 43, 17, 31, 60, 11, 18, 59, 54, 61, 7, 24, 39, 25, 13, 19, 42, 1, 3, 25, 41, 26, 4, 12, 50, 16, 50, 57, 36, 43, 1, 52, 1, 11, 37, 53, 20, 49, 35, 9, 22, 64, 53, 28, 34, 35, 19, 34, 7, 59, 46, 0, 49, 1, 16, 35, 40, 28, 9, 50, 12, 63, 36, 50, 58, 39, 53, 33, 12, 9, 29, 46, 0, 23, 50, 40, 45, 2]

Gbq:Rf?Vo&GHa$Aqwe AY.seESv;Fupw-LaMAGd $McN&?lDlsXe n ;YoHkW3JzoPVWGV-uh
k DWbP3l?yXltaoU?3Qh
Klbg!


In [44]:
# training loop
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
epochs = 10000
for epoch in range(epochs):
    # get a sample batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 4.554725170135498
Epoch: 2, Loss: 4.4788641929626465
Epoch: 3, Loss: 4.319748878479004
Epoch: 4, Loss: 4.405744552612305
Epoch: 5, Loss: 4.46208381652832
Epoch: 6, Loss: 4.471447467803955
Epoch: 7, Loss: 4.404010772705078
Epoch: 8, Loss: 4.39909553527832
Epoch: 9, Loss: 4.386376857757568
Epoch: 10, Loss: 4.4642720222473145
Epoch: 11, Loss: 4.264559268951416
Epoch: 12, Loss: 4.465758323669434
Epoch: 13, Loss: 4.438127040863037
Epoch: 14, Loss: 4.442635536193848
Epoch: 15, Loss: 4.425342082977295
Epoch: 16, Loss: 4.448538780212402
Epoch: 17, Loss: 4.490728855133057
Epoch: 18, Loss: 4.370786190032959
Epoch: 19, Loss: 4.360520839691162
Epoch: 20, Loss: 4.457721710205078
Epoch: 21, Loss: 4.436709403991699
Epoch: 22, Loss: 4.345595836639404
Epoch: 23, Loss: 4.5301666259765625
Epoch: 24, Loss: 4.360943794250488
Epoch: 25, Loss: 4.385802745819092
Epoch: 26, Loss: 4.373006820678711
Epoch: 27, Loss: 4.456490993499756
Epoch: 28, Loss: 4.393813610076904
Epoch: 29, Loss: 4.455334186

In [46]:
# testing the generate func
test_idx = torch.zeros((1,1), dtype=torch.long)
generate_op = m.generate(test_idx, max_new_tokens=100)[0].tolist()
print(decode(generate_op))


NIf here wirnart aun
Weron e y! ur fer loufisokie VO:
NGof m cathe ick'sit;
HAhaileyotay stoclllld w
